In [1]:
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


In [ ]:
# from utils.df_handle import *
import pendulum
from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator

local_tz = pendulum.timezone("Asia/Bangkok")

name='ACREGIS'
prefix='SYNC_'
csv_path = '/usr/local/airflow/plugins'+'/'

dag_params = {
    'owner': 'airflow',
    "depends_on_past": False,
    'start_date': datetime(2022, 5, 10, tzinfo=local_tz),
    'email_on_failure': True,
    'email_on_retry': False,
    'email':['duyvq@merapgroup.com', 'vanquangduy10@gmail.com'],
    'do_xcom_push': False,
    'execution_timeout':timedelta(seconds=300)
    # 'retries': 3,
    # 'retry_delay': timedelta(minutes=10),
}

dag = DAG(prefix+name,
          catchup=False,
          default_args=dag_params,
          schedule_interval= '*/30 8-17,23-23 * * *',
          tags=[prefix+name, 'Sync', '30mins']
)

In [8]:
start_date = '2022-04-01'
datenow = datetime.now().strftime("%Y-%m-%d")
datenow_mns1 = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
fdom = datetime.now().replace(day=1).strftime("%Y-%m-%d")

In [3]:
from_tb = "OM_AccumulatedRegis"

In [4]:
table_name = "d_accumulatedregis"
table_temp = "d_accumulatedregis_temp"

In [5]:
sql = \
f"""
DECLARE @from DATE = '2022-04-01'
DECLARE @to DATE = '2022-01-31'
select
SlsperID,
AccumulateID,
BranchID,
CustID,
LevelID,
[Status],
Crtd_DateTime,
Crtd_User,
Crtd_Prog,
LUpd_DateTime,
LUpd_User,
LUpd_Prog,
PurchaseAgreementValue,
PurchaseAgreementID,
EffectDateNbr
from {from_tb} where 
AccumulateID in ('CSBH2204OTC-14QD/MR-KS-STO','CSBH2102PNPP-04QD/PN-Q2')
and cast(Crtd_DateTime as date) < @from
and Status = 'C'
"""

In [6]:
df = get_ms_df(sql)
df['inserted_at'] = datetime.now()
bq_values_insert(df, f"{table_temp}", 3)
sql = \
f"""
DROP TABLE IF EXISTS biteam.{table_name};
CREATE TABLE biteam.{table_name} LIKE biteam.{table_temp}
PARTITION BY DATE(crtd_datetime)
CLUSTER BY branchid,custid
"""
execute_bq_query(sql)

1it [00:02,  2.95s/it]


In [7]:
bq_values_insert(df, f"{table_name}", 2)

1it [00:11, 11.12s/it]


In [11]:
usql = \
f"""
DECLARE @from DATE = '{start_date}'
DECLARE @to DATE = '2022-01-31'
select
SlsperID,
AccumulateID,
BranchID,
CustID,
LevelID,
[Status],
Crtd_DateTime,
Crtd_User,
Crtd_Prog,
LUpd_DateTime,
LUpd_User,
LUpd_Prog,
PurchaseAgreementValue,
PurchaseAgreementID,
EffectDateNbr
from {from_tb} where 
AccumulateID in ('CSBH2204OTC-14QD/MR-KS-STO','CSBH2102PNPP-04QD/PN-Q2')
and cast(Crtd_DateTime as date) >= @from
and Status = 'C'
"""

In [12]:
print(usql)


DECLARE @from DATE = '2022-04-01'
DECLARE @to DATE = '2022-01-31'
select
SlsperID,
AccumulateID,
BranchID,
CustID,
LevelID,
[Status],
Crtd_DateTime,
Crtd_User,
Crtd_Prog,
LUpd_DateTime,
LUpd_User,
LUpd_Prog,
PurchaseAgreementValue,
PurchaseAgreementID,
EffectDateNbr
from OM_AccumulatedRegis where 
AccumulateID in ('CSBH2204OTC-14QD/MR-KS-STO','CSBH2102PNPP-04QD/PN-Q2')
and cast(Crtd_DateTime as date) >= @from
and Status = 'C'



In [10]:
# table_name = "sync_dms_dv"
# table_temp = "sync_dms_dv_temp"

In [15]:
# print(sql)
df = get_ms_df(usql)
df['inserted_at'] = datetime.now()

dsql = \
f"""
delete from biteam.{table_name} where date(crtd_datetime) >= '{start_date}'
"""
print("delete_sql: ", dsql)
execute_bq_query(dsql)

bq_values_insert(df, f"{table_name}", 2)

delete_sql:  
delete from biteam.d_accumulatedregis where date(crtd_datetime) >= '2022-04-01'



1it [00:12, 12.47s/it]


In [ ]:
# dummy_start = DummyOperator(task_id="dummy_start", dag=dag)

# insert = PythonOperator(task_id="insert", python_callable=insert, dag=dag)

# update = PythonOperator(task_id="update", python_callable=update, dag=dag)

# dummy_start >> insert >> update